# Import Statements and Initialisation

In [2]:
%run "..\Python Client Connector\Python Client Connnector V1".py
%load_ext autoreload
%autoreload
%matplotlib notebook
import pandas as pd
from functools import partial
import threading
print(threading.active_count())
print(threading.enumerate())
#Create connector instance, with particular port for PUSH, PULL and SUB
connector = MTConnector()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
5
[<_MainThread(MainThread, started 16072)>, <Thread(Thread-4, started daemon 15412)>, <Heartbeat(Thread-5, started daemon 13700)>, <HistorySavingThread(IPythonHistorySavingThread, started 15356)>, <ParentPollerWindows(Thread-3, started daemon 6548)>]
[INIT] Ready to send commands to METATRADER (PUSH): 32768
[INIT] Listening for responses from METATRADER (PULL): 32769
[INIT] Listening for market data from METATRADER (SUB): 32770


# Subscribing to Tick Data

In [3]:
connector.subscribeToSymbolTicks()
print(threading.active_count())
print(threading.enumerate())

[KERNEL] Subscribed to  BID/ASK updates. See self.MARKET_DATA_DB.
6
[<_MainThread(MainThread, started 16072)>, <Thread(Thread-4, started daemon 15412)>, <Heartbeat(Thread-5, started daemon 13700)>, <HistorySavingThread(IPythonHistorySavingThread, started 15356)>, <ParentPollerWindows(Thread-3, started daemon 6548)>, <Thread(Thread-6, started 9404)>]


In [4]:
connector.MARKET_DATA_DB

{}

In [ ]:
connector._DWX_MTX_UNSUBSCRIBE_MARKETDATA_('GBPUSD')

In [ ]:
tickdataframe = pd.DataFrame(connector._Market_Data_DB['GBPUSD'])

In [ ]:
tickdataframe = tickdataframe[['Time','Bid','Ask','Flag']]
tickdataframe

In [ ]:
from pandas import HDFStore
hdf = HDFStore('testTickData.h5')

In [ ]:
hdf.put('testdata',tickdataframe,format='table',data_columns=True)

In [ ]:
hdf['testdata']

# Loading data from HD5 File

In [ ]:
readDataFrame = pd.read_hdf('testTickData.h5')
readDataFrame.set_index('Time').loc[:,'Bid':'Ask']
#truncatedDataFrame = readDataFrame
#truncatedDataFrame['Time'] = truncatedDataFrame['Time'].dt.floor('S')
#truncatedDataFrame

# Calculate OHLC Data

In [ ]:
OHLCData = readDataFrame.resample('1min').agg({'Bid':'ohlc','Ask':'ohlc','Flag':'max'})
OHLCData

In [ ]:
OHLCData.loc[:,'Bid'].plot()